In [1]:
# Setup

import numpy as np

# Ein Zufalls-Seed für Reproduzierbarkeit
np.random.seed(42)

from datetime import datetime
import pandas as pd
import seaborn as sns
from scipy import stats
import sklearn as sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, classification_report
from sklearn.impute import KNNImputer
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks, utils

sns.set_theme()

print(f"GPUs für die Berechnung: {len(tf.config.experimental.list_physical_devices('GPU'))}")

/home/nico/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nico/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nico/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nico/anaconda3/envs/tensorflow/lib

GPUs für die Berechnung: 0


/home/nico/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nico/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nico/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nico/anaconda

In [2]:
# OTU Metadaten einlesen
df = pd.read_csv("NIHMS841832-supplement-1.csv", sep=',')

# Ergebnisse des Feature Tables einlesen
feature = pd.read_csv('feature_table_otu.txt', sep='\t').T
feature = feature[1:][:-1]

In [3]:
# Gesunde Kontrollgruppe
HC = df[df.ibd_subtype.eq("HC")]

y = []
for row in feature.index:
    if any(True for val in HC['sample_name'] if val == row):
        y.append(1)
    else:
        y.append(0)

X = feature.iloc[:, :].values

In [4]:
feature.head()

,0,1,2,3,4,5,6,7,8,9,...,3101,3102,3103,3104,3105,3106,3107,3108,3109,3110
1629.SubjectIBD335,34292.0,20670.0,18413.0,9981.0,7071.0,6881.0,5411.0,5335.0,5289.0,4741.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1629.SubjectIBD643,15243.0,64328.0,0.0,0.0,0.0,4.0,4507.0,3216.0,15630.0,199.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1629.SubjectIBD539,22182.0,21589.0,0.0,1365.0,0.0,11501.0,33619.0,3638.0,5053.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1629.SubjectIBD078,0.0,805.0,0.0,0.0,0.0,4.0,330.0,2305.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1629.SubjectIBD671,0.0,19734.0,0.0,0.0,0.0,0.0,215.0,0.0,0.0,699.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
def nn(random_state):
    # Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = random_state)

    # Das Modell soll aufhören zu rechnen, falls es keine nennenswerten Verbesserungen mehr gibt
    early_stopping = callbacks.EarlyStopping(
        min_delta=0.001,
        patience=64,
        restore_best_weights=True
    )

    # Das NN besteht aus einer Mischung von Dense-, Normalization- und Dropout-Layern.
    # Dropout führt allem Anschein nach zu schlechterem F1
    # Weniger LUs führen zu besseren Ergebnissen
    network = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=[X_train.shape[1]]),
        #layers.Dropout(rate=0.5),
        layers.Dense(64, activation='relu'),
        #layers.Dropout(rate=0.5),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid'),
    ])

    # NN kompilieren
    network.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=[keras.metrics.Accuracy()]
    )

    # NN trainieren
    history = network.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        batch_size=64,
        epochs=512,
        callbacks=[early_stopping],
        verbose=0
    )

    # Scores berechnen
    # history_df = pd.DataFrame(history.history)

    y_pred = np.floor(network.predict(X_test))
    report = sklearn.metrics.classification_report(y_test, y_pred)
    f1 = np.round(f1_score(y_test, y_pred, average='macro'), 4)
    # confusion_matrix = sklearn.metrics.confusion_matrix(y_test, y_pred)
    # print(confusion_matrix)
    # print(report)
    return f1, report, network

    # history_df.loc[5:, ['loss', 'val_loss']].plot()

In [6]:
results = {}

for i in np.arange(100):
    f1, report, model = nn(i)
    results[i] = [model, f1]
    print(report)

df_results = pd.DataFrame.from_dict(results, orient='index', columns=['model', 'f1'])

df_results

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
              precision    recall  f1-score   support

           0       0.95      0.98      0.97       123
           1       0.80      0.57      0.67        14

    accuracy                           0.94       137
   macro avg       0.88      0.78      0.82       137
weighted avg       0.94      0.94      0.94       137

              precision    recall  f1-score   support

           0       0.98      0.99      0.98       131
           1       0.75      0.50      0.60         6

    accuracy                           0.97       137
   macro avg       0.86      0.75      0.79       137
weighted avg       0.97      0.97      0.97       137

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       125
           1       0.78  

              precision    recall  f1-score   support

           0       0.93      0.98      0.96       119
           1       0.82      0.50      0.62        18

    accuracy                           0.92       137
   macro avg       0.87      0.74      0.79       137
weighted avg       0.91      0.92      0.91       137

              precision    recall  f1-score   support

           0       0.98      0.99      0.98       122
           1       0.92      0.80      0.86        15

    accuracy                           0.97       137
   macro avg       0.95      0.90      0.92       137
weighted avg       0.97      0.97      0.97       137

              precision    recall  f1-score   support

           0       0.96      0.99      0.98       128
           1       0.80      0.44      0.57         9

    accuracy                           0.96       137
   macro avg       0.88      0.72      0.77       137
weighted avg       0.95      0.96      0.95       137

              preci

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       126
           1       0.75      0.55      0.63        11

    accuracy                           0.95       137
   macro avg       0.86      0.76      0.80       137
weighted avg       0.94      0.95      0.95       137

              precision    recall  f1-score   support

           0       0.94      0.98      0.96       119
           1       0.85      0.61      0.71        18

    accuracy                           0.93       137
   macro avg       0.89      0.80      0.84       137
weighted avg       0.93      0.93      0.93       137

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       123
           1       0.75      0.64      0.69        14

    accuracy                           0.94       137
   macro avg       0.85      0.81      0.83       137
weighted avg       0.94      0.94      0.94       137

              preci

              precision    recall  f1-score   support

           0       0.96      0.96      0.96       126
           1       0.55      0.55      0.55        11

    accuracy                           0.93       137
   macro avg       0.75      0.75      0.75       137
weighted avg       0.93      0.93      0.93       137

              precision    recall  f1-score   support

           0       0.95      0.98      0.96       125
           1       0.67      0.50      0.57        12

    accuracy                           0.93       137
   macro avg       0.81      0.74      0.77       137
weighted avg       0.93      0.93      0.93       137

              precision    recall  f1-score   support

           0       0.98      0.99      0.98       126
           1       0.89      0.73      0.80        11

    accuracy                           0.97       137
   macro avg       0.93      0.86      0.89       137
weighted avg       0.97      0.97      0.97       137

              preci

KeyError: 0

In [47]:
df_results_sorted = df_results.sort_values(by=['f1'], ascending=False)

for i in np.arange(10):
    df_results_sorted.iloc[i].model.save(f'models/best/model{i}.h5')

In [ ]:

for i in np.arange(1000):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = np.random.randint(100000))

    y_pred = np.floor(df_results_sorted.iloc[0].model.predict(X_test))
    report = sklearn.metrics.classification_report(y_test, y_pred)
    f1 = np.round(f1_score(y_test, y_pred, average='macro'), 4)
    confusion_matrix = sklearn.metrics.confusion_matrix(y_test, y_pred)
    print(confusion_matrix)
    print(report)
    print('\n\n\n')

[[128   0]
 [  1   8]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       128
           1       1.00      0.89      0.94         9

    accuracy                           0.99       137
   macro avg       1.00      0.94      0.97       137
weighted avg       0.99      0.99      0.99       137





[[125   0]
 [  3   9]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       125
           1       1.00      0.75      0.86        12

    accuracy                           0.98       137
   macro avg       0.99      0.88      0.92       137
weighted avg       0.98      0.98      0.98       137





[[121   0]
 [  2  14]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       121
           1       1.00      0.88      0.93        16

    accuracy                           0.99       137
   macro avg       0.99      0.94      0.96       1

[[122   0]
 [  3  12]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       122
           1       1.00      0.80      0.89        15

    accuracy                           0.98       137
   macro avg       0.99      0.90      0.94       137
weighted avg       0.98      0.98      0.98       137





[[126   0]
 [  1  10]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       126
           1       1.00      0.91      0.95        11

    accuracy                           0.99       137
   macro avg       1.00      0.95      0.97       137
weighted avg       0.99      0.99      0.99       137





[[126   0]
 [  1  10]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       126
           1       1.00      0.91      0.95        11

    accuracy                           0.99       137
   macro avg       1.00      0.95      0.97       1

[[126   0]
 [  0  11]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       126
           1       1.00      1.00      1.00        11

    accuracy                           1.00       137
   macro avg       1.00      1.00      1.00       137
weighted avg       1.00      1.00      1.00       137





[[123   0]
 [  1  13]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       123
           1       1.00      0.93      0.96        14

    accuracy                           0.99       137
   macro avg       1.00      0.96      0.98       137
weighted avg       0.99      0.99      0.99       137





[[126   0]
 [  0  11]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       126
           1       1.00      1.00      1.00        11

    accuracy                           1.00       137
   macro avg       1.00      1.00      1.00       1

[[132   0]
 [  0   5]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       132
           1       1.00      1.00      1.00         5

    accuracy                           1.00       137
   macro avg       1.00      1.00      1.00       137
weighted avg       1.00      1.00      1.00       137





[[129   0]
 [  1   7]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       129
           1       1.00      0.88      0.93         8

    accuracy                           0.99       137
   macro avg       1.00      0.94      0.96       137
weighted avg       0.99      0.99      0.99       137





[[127   0]
 [  0  10]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       127
           1       1.00      1.00      1.00        10

    accuracy                           1.00       137
   macro avg       1.00      1.00      1.00       1

[[124   0]
 [  1  12]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       124
           1       1.00      0.92      0.96        13

    accuracy                           0.99       137
   macro avg       1.00      0.96      0.98       137
weighted avg       0.99      0.99      0.99       137





[[128   0]
 [  1   8]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       128
           1       1.00      0.89      0.94         9

    accuracy                           0.99       137
   macro avg       1.00      0.94      0.97       137
weighted avg       0.99      0.99      0.99       137





[[122   0]
 [  2  13]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       122
           1       1.00      0.87      0.93        15

    accuracy                           0.99       137
   macro avg       0.99      0.93      0.96       1

...

[[127   0]
 [  1   9]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       127
           1       1.00      0.90      0.95        10

    accuracy                           0.99       137
   macro avg       1.00      0.95      0.97       137
weighted avg       0.99      0.99      0.99       137





[[127   0]
 [  1   9]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       127
           1       1.00      0.90      0.95        10

    accuracy                           0.99       137
   macro avg       1.00      0.95      0.97       137
weighted avg       0.99      0.99      0.99       137





[[131   0]
 [  0   6]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       131
           1       1.00      1.00      1.00         6

    accuracy                           1.00       137
   macro avg       1.00      1.00      1.00       1

[[126   0]
 [  3   8]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       126
           1       1.00      0.73      0.84        11

    accuracy                           0.98       137
   macro avg       0.99      0.86      0.92       137
weighted avg       0.98      0.98      0.98       137





[[123   0]
 [  2  12]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       123
           1       1.00      0.86      0.92        14

    accuracy                           0.99       137
   macro avg       0.99      0.93      0.96       137
weighted avg       0.99      0.99      0.98       137





[[129   0]
 [  0   8]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       129
           1       1.00      1.00      1.00         8

    accuracy                           1.00       137
   macro avg       1.00      1.00      1.00       1

[[125   0]
 [  1  11]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       125
           1       1.00      0.92      0.96        12

    accuracy                           0.99       137
   macro avg       1.00      0.96      0.98       137
weighted avg       0.99      0.99      0.99       137





[[126   0]
 [  2   9]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       126
           1       1.00      0.82      0.90        11

    accuracy                           0.99       137
   macro avg       0.99      0.91      0.95       137
weighted avg       0.99      0.99      0.98       137





[[123   0]
 [  1  13]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       123
           1       1.00      0.93      0.96        14

    accuracy                           0.99       137
   macro avg       1.00      0.96      0.98       1

[[125   0]
 [  2  10]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       125
           1       1.00      0.83      0.91        12

    accuracy                           0.99       137
   macro avg       0.99      0.92      0.95       137
weighted avg       0.99      0.99      0.98       137





[[124   0]
 [  2  11]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       124
           1       1.00      0.85      0.92        13

    accuracy                           0.99       137
   macro avg       0.99      0.92      0.95       137
weighted avg       0.99      0.99      0.98       137





[[126   0]
 [  1  10]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       126
           1       1.00      0.91      0.95        11

    accuracy                           0.99       137
   macro avg       1.00      0.95      0.97       1

[[121   0]
 [  2  14]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       121
           1       1.00      0.88      0.93        16

    accuracy                           0.99       137
   macro avg       0.99      0.94      0.96       137
weighted avg       0.99      0.99      0.98       137





[[125   0]
 [  0  12]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       125
           1       1.00      1.00      1.00        12

    accuracy                           1.00       137
   macro avg       1.00      1.00      1.00       137
weighted avg       1.00      1.00      1.00       137





[[126   0]
 [  0  11]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       126
           1       1.00      1.00      1.00        11

    accuracy                           1.00       137
   macro avg       1.00      1.00      1.00       1